# Reading Data and Create Prediction

This Version uses V002 of the prepared Data which you can find in the bscw (not publicly available during the time of the competition)

at first import the necessary libs:

In [ ]:
import pandas as pd
from sklearn import ensemble
from sklearn import svm
from sklearn import preprocessing
import numpy as np
import xgboost as xgb

read the data from file, obviously you should adjust the path on your system:

In [ ]:
trainOrigin = pd.read_csv('~/Documents/DMC16/dataPrep/DataPrepV002Train.csv')
test = pd.read_csv('~/Documents/DMC16/dataPrep/DataPrepV002Test.csv')

at first we need to define the methods we want to use later, so here we define the preprocess method.

some hints:
- attributetodrop is a list of attributes that i just drop. They are all of the type String and for a first working draft it doesn't make sense to transform all of them to something the algorithms can handle. This should be done later.

-  attributetofillna: the empty values of these attributes will be filled. will be fixed with the next version of the dataset hopefully.

- all rows with null in the original dataset are ignored for this model

- LabelEncoder is similar to categoryToNumber in Knime. should be improved later, because paymentMethod should not be numeric


In [ ]:
def preprocess(train, test):
    attributetodrop = ['row ID', 'BasketID', 'customerID', 'Buy_First', 'Buy_Last', 'orderID','articleID','sizeCode',
                      'voucherID','price [Binned]','BasketValue [Binned]','SinglePrice [Binned]','orderDate',
                      'RealArticleID','RealCustOrderDayID','rrp [Binned]']
    attributetofillnan= ['Mehrfachbesteller','ExtremeBesteller','Einmalbesteller']
    train = train.drop(attributetodrop, axis=1)
    test = test.drop(attributetodrop, axis=1)

    print("train.info()")
    print(train.info())

    train= train[train.rrp.notnull()]
    test= test[test.rrp.notnull()]
    
    #fillNa with 0
    for attr in attributetofillnan:
        train[attr]=train[attr].fillna(0)
        test[attr]=test[attr].fillna(0)

    le = preprocessing.LabelEncoder()
    train['paymentMethod'] = le.fit_transform(train['paymentMethod'])
    test['paymentMethod'] = le.transform(test['paymentMethod'])


    print(train.info())

    return train, test


and then we need to call the method:

In [ ]:
train, test = preprocess(trainOrigin, test)

After reading an preparing the data we need to remove the target variable form the dataset

In [ ]:
print(train.info())
label = train[['returnQuantity']].values.ravel()
train=train.drop(['returnQuantity'],axis=1)
print(train.info())

labelTest = test[['returnQuantity']].values.ravel()
test=test.drop(['returnQuantity'],axis=1)

now it's time to create a prediction

In [ ]:
def predict(train,label, test):
    print("train")
    #clf = xgb.XGBClassifier(max_depth=3, n_estimators=100,learning_rate=0.05).fit(train,label)

    clf = ensemble.RandomForestClassifier(n_estimators=100, n_jobs=2,)
    #clf = svm.LinearSVC()
    clf.fit(train, label)
    print("classify")
    pred = clf.predict(test)
    #test['prediction']=pred
    #predictions = gbm.predict(test)
    return pred

and run it:

In [ ]:
print("train and predict")
pred = predict(train,label,test)

in the end we want to know how good we are and therefore need an evaluation:

In [ ]:
def evaluate(pred,label):
    summe=0
    rowcount=0
    print("len Predictions")
    print(len(pred))
    for row in range(len(pred)):
        summe=summe+abs(pred[row]-label[row])
        rowcount=rowcount+1
    print("summe Fehler",str(summe))
    print("summe rowCount",str(rowcount))

    return summe/rowcount

In [ ]:
print("evaluate")
print(evaluate(pred,labelTest))


That's it :-)